Сегодня мы поговорим о том, как доставать из сырого текста связи между объектами

## предустановка всего необходимого

In [1]:
# поставим Spacy, оттуда будем загружать модели NER
# ! pip3 install -U spacy==2.1.0

!pip3 install spacy
!python3 -m spacy download en

You should consider upgrading via the '/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/bin/python -m pip install --upgrade pip' command.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 5.5 MB/s            
You should consider upgrading via the '/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
!pip3 list

Package              Version
-------------------- ------------------
appdirs              1.4.4
appnope              0.1.2
argon2-cffi          21.3.0
argon2-cffi-bindings 21.2.0
attrs                21.2.0
backcall             0.2.0
beautifulsoup4       4.10.0
bleach               4.1.0
blis                 0.7.5
bs4                  0.0.1
catalogue            2.0.6
certifi              2021.10.8
cffi                 1.15.0
charset-normalizer   2.0.9
click                8.0.3
cssselect            1.1.0
cycler               0.11.0
cymem                2.0.6
DAWG-Python          0.7.2
debugpy              1.5.1
decorator            5.1.0
defusedxml           0.7.1
docopt               0.6.2
en-core-web-sm       3.2.0
entrypoints          0.3
fake-useragent       0.1.11
fonttools            4.28.5
graphviz             0.19.1
idna                 3.3
imageio              2.13.5
importlib-metadata   4.10.0
importlib-resources  5.4.0
ipykernel            6.6.0
ipython              7.30.1
i

In [3]:
# сделаем еще импорты нужных библиотек

from tqdm.notebook import tqdm # визуализация прогресс-баров

import re # регулярки
from spacy.lang.en import English

# для датафреймов
import pandas as pd
import numpy as np

# визуализации
%matplotlib inline

# импортируем Spacy и нужные части оттуда
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm") # загрузили обученную ML-модель



## Шаг 1: из сырого текста вытащим упоминание сущностей
*(такая задача называется Named Entity Recognition  или NER)*

In [50]:
# text = "When Sebastian Thrun started working on self-driving cars at Google in 2007,\
#  few people outside of the company took him seriously."

text = "I ate two chocolate Mars bars and one Twix."

doc = nlp(text) # обрабатываем текст
displacy.render(doc, style = "ent", jupyter = True)

[здесь подробнее про  параметры визуализации](https://spacy.io/usage/visualizers)

In [5]:
# посмотрим на список сущностей в документе
doc.ents

(Sebastian Thrun, Google, 2007)

## экспериментальная часть: попробуем на большом датасете

*(давайте его подготовим)*

In [ ]:
# !unzip elonmusk.csv.zip # запустите ячейку если загружаете файл в заархивированном виде

In [21]:
# откроем датафрейм
df = pd.read_csv("Twitter- Oscars(2).csv", encoding="latin-1")

In [24]:
df.columns = df.iloc[0]
df.head(3)

,Date,Screen Name,Full Name,Tweet Text,Tweet ID,App,Followers,Follows,Retweets,Favorites,Verfied,User Since,Location,Bio,Profile Image,Google Maps
0,Date,Screen Name,Full Name,Tweet Text,Tweet ID,App,Followers,Follows,Retweets,Favorites,Verfied,User Since,Location,Bio,Profile Image,Google Maps
1,2/27/2017 23:48:13,@caileonn,???i?i? ?eo??,"RT @deray: How Far I'll Go, Moana. feat. Auli'...",836452929183731712,Twitter for iPhone,397,295,6777,0,No,8/16/2009,West Coast,{ Instagram: @caileonn },View,NaN
2,2/27/2017 23:48:13,@luisdemdiaz,Luis Diaz,RT @cosmos212: Será la foto de la década?? #os...,836452930601484288,Twitter Web Client,67,119,77,0,No,2/23/2016,"Culiacán, Sinaloa",25 años.,View,NaN


In [25]:
df.tweet[-2:] # посмотрим на содержимое колонки с твитами

AttributeError: 'DataFrame' object has no attribute 'tweet'

In [52]:
df_small = df[["Tweet Text","Screen Name"]].rename(columns={"Tweet Text":"text","Screen Name":"name"}).drop(0).reset_index()

df_small.head(5)

,index,text,name
0,1,"RT @deray: How Far I'll Go, Moana. feat. Auli'...",@caileonn
1,2,RT @cosmos212: Será la foto de la década?? #os...,@luisdemdiaz
2,3,RT @Muh_Sadam: #Oscars Ryan Gosling & Justin T...,@tamilu40
3,4,RT @Essence: Class. Excellence. Unmatched. #De...,@mariaglow
4,5,RT @rayliotta: Just heard what happened last n...,@LostinMemphis


In [18]:
print(len(df.tweet.to_list()))
elon = df.tweet.to_list()

9286


In [45]:
oscar = df_small.text.to_list()

In [51]:
# функция которая покажет твиты с сущностями и кореференцией

for doc in oscar[100:105]: # для твитов выбранного диапазона (его можно менять)

    if len(doc) > 50: # если длина твита больше 50 символов
        doc = nlp(doc) # обрабатываем текст spacy-функцией

        if True: #len(doc.ents) >0: # если найдены именованные сущности
            displacy.render(doc, style = "ent",jupyter =True) # отображаем такие твиты 
            print('\nEntities found:',doc.ents) # покажите сами сущности тоже



Entities found: (RT @KarmaJonez,)


/Users/Alexey.Zhebel/IdeaProjects/CompLing/venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Entities found: ()



Entities found: (https://t.co/OhHkMRs9kN v,)



Entities found: (RT @girlposts, jimmy kimmel, https://t.co)



Entities found: (RT @neontaster, #BestPicture #)


# Как еще можно извлекать информацию?
- не через сущности, а через POS-тэги, как мы делали на прошлых парах([1](https://colab.research.google.com/drive/1l4MUvWtQFpRfrl-afGKKbuFTEJ0-2JI4?usp=sharing),  [2](https://colab.research.google.com/drive/1WaO4hQxzqhPA5Lvc23bPLKM3O6smH4iA?usp=sharing))

- из русскоязычных текстов, при помощи [natasha](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/NER.ipynb)

- с помощью [этой библиотеки](https://github.com/fhamborg/Giveme5W1H) (работает с json-файлами)

- можно построить [граф знаний в neo4j](http://blog.bruggen.com/2016/03/the-neo4j-knowledge-graph.html)
